In [ ]:
#Name : Komal Dodiya
#Student ID : U01948955
#Subject : Data Science


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
daily_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/New_A_SN_d_tot_V2.0.csv', sep=';')

In [ ]:
daily_df.head()

In [ ]:
daily_df

In [ ]:
daily_df.info()

In [ ]:
daily_df.describe()

In [ ]:
daily_df.isnull().sum()

In [ ]:
# Function to determine the time unit and create a datetime column
def create_datetime_column(df):
    # Check for year, month, and day columns
    potential_date_cols = ['year', 'month', 'day']  # Initialize with common names

    # Find actual column names in df that resemble year, month, day
    for col in df.columns:
      if "year" in col.lower():
        potential_date_cols[0] = col
      if "month" in col.lower():
        potential_date_cols[1] = col
      if "day" in col.lower():
        potential_date_cols[2] = col

    # Check if all date components exist
    if all(col in df.columns for col in potential_date_cols):
      try:
        df['datetime'] = pd.to_datetime(df[potential_date_cols])
        return df
      except:
        print("Error creating datetime column. Check data types in the date columns.")
        return df  # Return original DataFrame if error occurs
    else:
      print("Columns 'year', 'month', and 'day' (or similar) not found in the DataFrame.")
      return df


import pandas as pd

daily_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/New_A_SN_d_tot_V2.0.csv', sep=';')
daily_df = create_datetime_column(daily_df)

daily_df.head()

In [ ]:
!pip install prophet

from prophet import Prophet

# Prepare the data for Prophet
daily_df_prophet = daily_df[['datetime', 'total_sunspots']].rename(columns={'datetime': 'ds', 'total_sunspots': 'y'})

# Create and fit the Prophet model
model = Prophet()
model.fit(daily_df_prophet)

# Create future dates for prediction
future = model.make_future_dataframe(periods=365, freq='D') # Predict for 365 days

# Make predictions
forecast = model.predict(future)

# Print predictions for 100, 200, and 365 days into the future
print("Sunspot Predictions:")
for days in [100, 200, 365]:
  predicted_sunspots = forecast.iloc[-days]['yhat']
  print(f"{days} days into the future: {predicted_sunspots}")

# Plot the forecast
fig1 = model.plot(forecast)
# Plot the components of the forecast
fig2 = model.plot_components(forecast)

In [ ]:

import pandas as pd

daily_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/New_A_SN_d_tot_V2.0.csv', sep=';')
daily_df = create_datetime_column(daily_df)
# Prepare the data for Prophet with yearly frequency
yearly_df = daily_df.groupby(pd.Grouper(key='datetime', freq='Y')).sum().reset_index()
yearly_df_prophet = yearly_df[['datetime', 'total_sunspots']].rename(columns={'datetime': 'ds', 'total_sunspots': 'y'})

# Create and fit the Prophet model
yearly_model = Prophet()
yearly_model.fit(yearly_df_prophet)

# Create future dates for prediction (in years)
future_years = yearly_model.make_future_dataframe(periods=20, freq='Y')

# Make predictions
forecast_years = yearly_model.predict(future_years)

# Print predictions for 1, 10, and 20 years into the future
print("\nSunspot Predictions (Yearly):")
for years in [1, 10, 20]:
  predicted_sunspots = forecast_years.iloc[-years]['yhat']
  print(f"{years} years into the future: {predicted_sunspots}")

# Plot the yearly forecast
fig5 = yearly_model.plot(forecast_years)
plt.title("Yearly Sunspot Predictions")
plt.xlabel("Date")
plt.ylabel("Total Sunspots")

# Plot the components of the yearly forecast
fig6 = yearly_model.plot_components(forecast_years)
plt.title("Yearly Sunspot Components")
plt.xlabel("Date")
plt.ylabel("Values")

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

# Prepare the data for Prophet
daily_df_prophet = daily_df[['datetime', 'total_sunspots']].rename(columns={'datetime': 'ds', 'total_sunspots': 'y'})

# Create and fit the Prophet model
model = Prophet()
model.fit(daily_df_prophet)

# Create future dates for prediction
future = model.make_future_dataframe(periods=365, freq='D') # Predict for 365 days

# Make predictions
forecast = model.predict(future)

# Print predictions for 100, 200, and 365 days into the future
print("Sunspot Predictions:")
for days in [100, 200, 365]:
  predicted_sunspots = forecast.iloc[-days]['yhat']
  print(f"{days} days into the future: {predicted_sunspots}")

# Plot the forecast
fig1 = model.plot(forecast)
# Plot the components of the forecast
fig2 = model.plot_components(forecast)

# Function to evaluate the model
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mape, r2

#with different growth settings
growth_settings = ['logistic', 'linear', 'flat']
for growth in growth_settings:
  model_growth = Prophet(growth=growth)

  if growth == 'logistic':
    capacity = daily_df_prophet['y'].max() * 1.1
    daily_df_prophet['cap'] = capacity
    future['cap'] = capacity
  model_growth.fit(daily_df_prophet)
  forecast_growth = model_growth.predict(future)
  if growth == 'logistic':
    daily_df_prophet = daily_df_prophet.drop('cap', axis=1)

  # Evaluate the model for the current growth setting
  mae_growth, mape_growth, r2_growth = evaluate_model(daily_df_prophet['y'][-365:], forecast_growth['yhat'][-365:]) # Evaluate using last 365 days
  print(f"\nGrowth: {growth}")
  print(f"MAE: {mae_growth}, MAPE: {mape_growth}, R2: {r2_growth}")

  print(f"Predictions for {growth} growth:")
  for days in [100, 200, 365]:
    predicted_sunspots = forecast_growth.iloc[-days]['yhat']
    print(f"{days} days into the future: {predicted_sunspots}")

#with different seasonalities
seasonalities = [{'period': 30, 'fourier_order': 5}, {'period': 365, 'fourier_order': 10}]
for season in seasonalities:
  m = Prophet()
  m.add_seasonality(name='custom_season', period=season['period'], fourier_order=season['fourier_order'])
  m.fit(daily_df_prophet)
  future_s = m.make_future_dataframe(periods=365, freq='D')
  forecast_s = m.predict(future_s)
  mae_s, mape_s, r2_s = evaluate_model(daily_df_prophet['y'][-365:], forecast_s['yhat'][-365:])
  print(f"\nSeasonality: period={season['period']}, fourier_order={season['fourier_order']}")
  print(f"MAE: {mae_s}, MAPE: {mape_s}, R2: {r2_s}")

  print(f"Predictions for custom seasonality:")
  for days in [100, 200, 365]:
      predicted_sunspots = forecast_s.iloc[-days]['yhat']
      print(f"{days} days into the future: {predicted_sunspots}")

#with different changepoint settings
changepoint_settings = [{'n_changepoints': 10, 'changepoint_prior_scale': 0.05},
                        {'n_changepoints': 25, 'changepoint_prior_scale': 0.5}]
for setting in changepoint_settings:
  m = Prophet(n_changepoints=setting['n_changepoints'], changepoint_prior_scale=setting['changepoint_prior_scale'])
  m.fit(daily_df_prophet)
  future_c = m.make_future_dataframe(periods=365, freq='D')
  forecast_c = m.predict(future_c)
  mae_c, mape_c, r2_c = evaluate_model(daily_df_prophet['y'][-365:], forecast_c['yhat'][-365:])
  print(f"\nChangepoint: n_changepoints={setting['n_changepoints']}, changepoint_prior_scale={setting['changepoint_prior_scale']}")
  print(f"MAE: {mae_c}, MAPE: {mape_c}, R2: {r2_c}")

  print(f"Predictions for changepoint settings:")
  for days in [100, 200, 365]:
      predicted_sunspots = forecast_c.iloc[-days]['yhat']
      print(f"{days} days into the future: {predicted_sunspots}")

In [ ]:
# Prepare the data for Prophet with yearly frequency
yearly_df = daily_df.groupby(pd.Grouper(key='datetime', freq='Y')).sum().reset_index()
yearly_df_prophet = yearly_df[['datetime', 'total_sunspots']].rename(columns={'datetime': 'ds', 'total_sunspots': 'y'})

# Define parameter settings to test
growth_settings = ['logistic', 'linear', 'flat']
seasonality_settings = [{'period': 10, 'fourier_order': 3}, {'period': 11, 'fourier_order': 5}] # Example seasonalities
changepoint_settings = [{'n_changepoints': 5, 'changepoint_prior_scale': 0.01}, {'n_changepoints': 10, 'changepoint_prior_scale': 0.1}]

# Store results for comparison
results = []

# Loop through each parameter combination
for growth in growth_settings:
    for seasonality in seasonality_settings:
        for changepoint in changepoint_settings:
            # Create and fit the Prophet model with the current settings
            model = Prophet(growth=growth,
                            n_changepoints=changepoint['n_changepoints'],
                            changepoint_prior_scale=changepoint['changepoint_prior_scale'])
            model.add_seasonality(name='custom_season', period=seasonality['period'], fourier_order=seasonality['fourier_order'])

            if growth == 'logistic':
                capacity = yearly_df_prophet['y'].max() * 1.1
                yearly_df_prophet['cap'] = capacity

            model.fit(yearly_df_prophet)

            # Create future_years DataFrame here, after potentially adding 'cap'
            future_years = model.make_future_dataframe(periods=20, freq='Y')

            if growth == 'logistic':
                future_years['cap'] = capacity # Add 'cap' to future_years for logistic growth

            forecast_years = model.predict(future_years)
            # Evaluate the model
            mae, mape, r2 = evaluate_model(yearly_df_prophet['y'], forecast_years['yhat'][:-20])
            predictions = [forecast_years['yhat'].iloc[-1], forecast_years['yhat'].iloc[-10], forecast_years['yhat'].iloc[-20]]

            results.append({
                'growth': growth,
                'seasonality': seasonality,
                'changepoint': changepoint,
                'mae': mae,
                'mape': mape,
                'r2': r2,
                'predictions': predictions  # Store predictions for 1, 10, 20 years
            })

# Display and analyze the results
for result in results:
    print(f"Growth: {result['growth']}, Seasonality: {result['seasonality']}, Changepoint: {result['changepoint']}")
    print(f"  MAE: {result['mae']:.2f}, MAPE: {result['mape']:.2f}, R2: {result['r2']:.2f}")
    print("  Predictions (1, 10, 20 years):", result['predictions'])

plt.show()